### Sprint 4 – Notebook 01: Comparación Baseline vs Modelo Actual

Este notebook analiza el desempeño del modelo baseline y del modelo actual optimizado.

Incluye:

- Carga de métricas generadas por los scripts del Sprint 4  
- Comparación tabular  
- Gráfico comparativo de métricas  
- Ordenamiento por delta  
- Comentarios y conclusiones  

Las rutas ya están ajustadas para ejecutarse desde:  
`notebooks/Sprint4/`


In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
# IMPORTANTE:
# El notebook está en notebooks/Sprint4/
# Por eso las rutas suben dos niveles (../../)

metrics_baseline_path = Path("../../models/sprint4/resultados/metrics_baseline.csv")
metrics_model_path = Path("../../models/sprint4/resultados/metrics_modelo_actual.csv")

metrics_baseline_path, metrics_model_path


In [ ]:
if not metrics_baseline_path.exists():
    raise FileNotFoundError(f"No existe {metrics_baseline_path}. Ejecuta 01_run_inference_baseline.py")

if not metrics_model_path.exists():
    raise FileNotFoundError(f"No existe {metrics_model_path}. Ejecuta 02_run_inference_model_actual.py")

mb = pd.read_csv(metrics_baseline_path)
mm = pd.read_csv(metrics_model_path)

print("Métricas baseline:")
display(mb)

print("Métricas modelo actual:")
display(mm)


In [ ]:
mb_dict = mb.iloc[0].to_dict()
mm_dict = mm.iloc[0].to_dict()

all_keys = sorted(set(mb_dict.keys()) | set(mm_dict.keys()))

rows = []
for k in all_keys:
    v_b = mb_dict.get(k, float("nan"))
    v_m = mm_dict.get(k, float("nan"))
    try:
        delta = float(v_m) - float(v_b)
    except:
        delta = float("nan")

    rows.append({
        "metrica": k,
        "baseline": v_b,
        "modelo_actual": v_m,
        "delta": delta,
    })

df_comp = pd.DataFrame(rows)
df_comp


In [ ]:
metricas_interes = [
    "accuracy",
    "precision_macro",
    "recall_macro",
    "f1_macro",
    "roc_auc"
]

df_plot = df_comp[df_comp["metrica"].isin(metricas_interes)].copy()
df_plot


In [ ]:
if not df_plot.empty:
    plt.figure(figsize=(8, 5))

    x = range(len(df_plot))
    width = 0.35

    plt.bar(
        [i - width/2 for i in x],
        df_plot["baseline"],
        width=width,
        label="Baseline"
    )
    plt.bar(
        [i + width/2 for i in x],
        df_plot["modelo_actual"],
        width=width,
        label="Modelo Actual"
    )

    plt.xticks(list(x), df_plot["metrica"], rotation=45)
    plt.ylabel("Valor de la métrica")
    plt.title("Comparación de métricas – Baseline vs Modelo Actual")
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No hay métricas para graficar.")


In [ ]:
df_comp_sorted = df_comp.sort_values("delta", ascending=False)
df_comp_sorted


#### Conclusiones preliminares

- `f1_macro` y `recall_macro` suelen ser las métricas más relevantes cuando existe desbalance.  
- `roc_auc` resume de forma global la capacidad discriminativa si está disponible.  
- La columna `delta` permite cuantificar claramente la mejora del modelo actual frente al baseline.  

Esta información se integra automáticamente en el informe generado por:

